In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

from sklearn.preprocessing import MinMaxScaler,StandardScaler,Imputer,LabelEncoder,OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, log_loss, classification_report)
from imblearn.over_sampling import SMOTE
import xgboost

# Import and suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Вывод таблицы с данными

In [ ]:
df = pd.read_csv('../input/WA_Fn-UseC_-HR-Employee-Attrition.csv')
df.head()

# Гендерное распределение сотрудников в наборе данных

Распределение по полу показывает, что набор данных показывает более высокую относительную долю бывших работников мужского пола, чем бывших работников женского пола

In [ ]:
print("Гендерное распределение бывших сотрудников в наборе данных: мужчины = {:.1f}%; женщины {:.1f}%.".format((df[(df['Attrition'] == 'Yes') & 
 (df['Gender'] == 'Male')].shape[0] / df[df['Gender'] == 'Male'].shape[0])*100,
 (df[(df['Attrition'] == 'Yes') & (df['Gender'] == 'Female')].shape[0] / df[df['Gender'] == 'Female'].shape[0])*100))

# Расстояние от дома до работы

Расстояние от дома до работы варьируется от 1 до 29 миль. Нет заметной сильной зависимости между DistanceFromHome и Attrition

In [ ]:
print("Расстояние от дома до работы от {} до {} миль.".format(df['DistanceFromHome'].min(),df['DistanceFromHome'].max()))
print('Среднее расстояние от дома для сотрудников без выгорания {:.2f} миль и без выгорания {:.2f} миль'.format(
    df[df['Attrition'] == 'No']['DistanceFromHome'].mean(), df[df['Attrition'] == 'Yes']['DistanceFromHome'].mean()))

У целевого столбца Attrition(который является категориальным) будем считать зачения 1 - yes, значения 0 - no

In [ ]:
df['Attrition']=df['Attrition'].map({'Yes':1,'No':0})

# Поиск нулеывых значений

In [ ]:
display(df.isnull().any())

# Краткий обзор набора данных

In [ ]:
df.info()

In [ ]:
df['Attrition'].value_counts()

# Итог:
- набор данных: 1470 наблюдений (строки), 35 объектов (переменных)
- нет отсутствующих данных
- у нас есть только два типа данных в этом наборе данных: категориальные и целые числа
- «Attrition» - целевая переменная
- несбалансированный набор данных: 1233 сотрудников не покинули организацию, в то время как 237 покинули организацию, сделав наш набор данных несбалансированным

# Визуализация распределения данных для каждой переменной

In [ ]:
df.hist(edgecolor='black', linewidth=1.2, figsize=(15, 15));

**Несколько наблюдений могут быть сделаны на основе информации и гистограмм для числовых признаков:**

- Многие гистограммы хвостовые(например, MonthlyIncome DistanceFromHome, YearsAtCompany)
- Распределение по возрасту - основная масса сотрудников составляет от 25 до 45 лет.
- EmployeeCount и StandardHours являются постоянными значениями для всех сотрудников. Они, вероятно, будут избыточными функциями.
- Номер сотрудника - это идентификатор для сотрудников 

# Описательная статистика

In [ ]:
df.describe()

Средний возраст сотрудников - 37 лет.
Большинство людей получают повышение в течение 2-5 лет
Среднее время работы в компании составляет 7 лет
Ни у кого нет рейтинга производительности ниже 3

# Визуализация целевого столбца

In [ ]:
 sns.countplot(x='Attrition',data=df)

Видно, что данные несбалансированы

# Визуализация категориальных столбцов

In [ ]:
sns.countplot(x='MaritalStatus',hue='Attrition',data=df)

Из MaritalStatus мы видим, что одинокие и женатые имеют больше истощения, чем разведенные.

In [ ]:
sns.countplot(x='BusinessTravel',hue='Attrition',data=df)

Люди, которые бывают в коммандировках редко, имеют более высокий уровень истощения.

In [ ]:
sns.countplot(x='Department',hue='Attrition',data=df)

Из Department у сотрудников научно-исследовательских и продаж больше истощения, у Human resourses значительно меньше истощения

In [ ]:
sns.countplot(x='Gender',hue='Attrition',data=df)

С точки зрения пола, у работников мужского пола больше истощения, чем у женщин

In [ ]:
plt.subplots(figsize=(20,5))
sns.countplot(x='JobRole',hue='Attrition',data=df)

Из ролей сотрудников выясняется, что сотрудники, которые являются руководителями отдела продаж и научными сотрудниками, имеют более высокий уровень истощения

In [ ]:
df['PerformanceRating'].value_counts() 
sns.countplot(x='PerformanceRating',hue='Attrition',data=df)

Сотрудники с рейтингом эффективности 3 и 4 имеют более высокий уровень истощения

In [ ]:
sns.countplot(x='RelationshipSatisfaction',hue='Attrition',data=df)

Истощение практически не зависит от уровня удволтеворенности семейными отношениями

In [ ]:
plt.figure(figsize=(15,15))
sns.lmplot("YearsAtCompany", "MonthlyIncome", data=df, size=10) 

Мы видим, что в компании много сотрудников, зарабатывающих более 10 тысяч в месяц, независимо от того, как долго они работают в компании. Но после 10-летней отметки происходит сокращение числа низкооплачиваемых работников, что повышает средние показатели по возрастным группам.

### Матрица корреляции

Построив матрицу корреляции, мы получим хороший обзор того, как функции связаны друг с другом.

# Заменяем категориальные признаки на числовые

In [ ]:
def transform(feature):
    le=LabelEncoder()
    df[feature]=le.fit_transform(df[feature])
    print(le.classes_)

In [ ]:
cat_df=df.select_dtypes(include='object')
cat_df.columns

In [ ]:
for col in cat_df.columns:
    transform(col)

In [ ]:
df.head()

In [ ]:
del df["Over18"]
del df["EmployeeNumber"]
del df["EmployeeCount"]
del df["StandardHours"]

In [ ]:
corr = df.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

In [ ]:
corr = df.corr().abs()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)


JobLevel тесно связан с Age, поскольку пожилые сотрудники обычно стремятся занять более высокие должности в компании.

MonthlyIncome очень сильно связан с JobLevel, как и ожидалось, поскольку старшие сотрудники определенно будут зарабатывать больше.

PerformanceRating тесно связан с PercentSalaryHike, что вполне очевидно.

TotalWorkingYears тесно связан с JobLevel, который ожидается, поскольку старшие сотрудники должны были работать в течение более длительного периода времени.

YearsWithCurrManager тесно связан с YearsAtCompany.

YearsAtCompany связана с YearsInCurrentRole.

Можно отбросить некоторые функции с высокой степенью корреляции, так как они добавляют избыточность к модели, но поскольку корреляция в общем случае очень мала можно пока оставить все функции.

# Машинное обучене

Наша цель - проверить, уволили ли работника или нет. Таким образом, мы принимаем Attrition в качестве целевого(y), а остальные функции в качестве переменных(X).
Оставим только те переменные, которые сильно коррелируют с целевой переменной Attrition

Более сильные показатели уходящих людей включают в себя:

MonthlyIncome: люди с более высокой заработной платой с меньшей вероятностью покидают компанию.

OverTime: люди, которые работают сверхурочно, чаще покидают компанию.

YearsWithCurrManager: Большое количество выпускников уходит через 6 месяцев после своих текущих менеджеров.

Возраст: работники в относительно молодом возрасте 25-35 лет чаще уходят.

TotalWorkingYears: более опытные сотрудники реже уходят. Сотрудники, у которых стаж от 5 до 8 лет - как потенциально имеющие более высокий риск уйти.

In [ ]:
data_features=['Age','EnvironmentSatisfaction','JobInvolvement', 
               'JobLevel','JobSatisfaction','MaritalStatus','MonthlyIncome',
               'OverTime','StockOptionLevel','TotalWorkingYears', 
               'YearsAtCompany','YearsInCurrentRole','YearsWithCurrManager']
label_data = df.copy()
X=label_data[data_features]
y=label_data.Attrition

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42,shuffle=True)
model = LogisticRegression()

In [ ]:
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

model.score(X_test,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred,labels=[0,1])
confusion_matrix

Вывод из матрицы ошибок: модель в основном предсказывала сотрудников, которые не были уволены - True Negative

Данная модель может всегда классифицировать каждого сотрудника как сотрудника компании и достигать точности около 80%. Кроме того, модель не сможет классифицировать тех сотрудников, которые действительно чувствуют выгорание относительно тех, кто чувствует себя отлично. Таким образом, необходимо устранить дисбаланс, использовав передискретизацию. При увеличении выборки меньшего класса до 50% выборки, точность до 80% становится реалистичней.


Используем алгоритм SMOTE (Synthetic Minority Oversampling Technique), который генерирует определенное количество похожих примеров класса с меньшим количеством данных, но при этом не дублирует данные в этом классе.

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE()

x1,y1 = sm.fit_sample(X,y)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(x1,y1,test_size=0.3,random_state=10)

model = LogisticRegression()

In [ ]:
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

model.score(X_test,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test,y_pred)
confusion_matrix

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

In [ ]:
#Библиотеки
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [ ]:
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)

In [ ]:
tree.score(X_test, y_test)